# Final Project Data Science Sanbercode 
## ivanmuhsiegfried@gmail.com

## Lakukan Importing

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import re
regex = re.compile(r"\[|\]|<", re.IGNORECASE)

from sklearn.model_selection import GridSearchCV 
from sklearn.model_selection import train_test_split

from sklearn.metrics import roc_auc_score as rocauc
from xgboost import XGBClassifier

## Baca Data

In [2]:
train_df = pd.read_csv('train.csv')
train_df.head()

,id,Umur,Kelas Pekerja,Berat Akhir,Pendidikan,Jmlh Tahun Pendidikan,Status Perkawinan,Pekerjaan,Jenis Kelamin,Keuntungan Kapital,Kerugian Capital,Jam per Minggu,Gaji
0,0,21,Wiraswasta,242912,SMA,9,Belum Pernah Menikah,Servis Lainnya,Perempuan,0.0,0.0,35.0,<=7jt
1,1,49,Wiraswasta,140782,10th,6,Cerai,Ekesekutif Managerial,Perempuan,0.0,0.0,40.0,<=7jt
2,2,44,Wiraswasta,120057,D3,12,Menikah,Ekesekutif Managerial,Laki2,61404000.0,0.0,45.0,>7jt
3,3,24,Wiraswasta,194630,Sarjana,13,Belum Pernah Menikah,Spesialis,Laki2,0.0,0.0,35.0,<=7jt
4,4,33,Wiraswasta,219619,Master,14,Menikah,Spesialis,Laki2,210336000.0,0.0,40.0,>7jt


## Periksa Null

In [3]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35994 entries, 0 to 35993
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     35994 non-null  int64  
 1   Umur                   35994 non-null  int64  
 2   Kelas Pekerja          35994 non-null  object 
 3   Berat Akhir            35994 non-null  int64  
 4   Pendidikan             35994 non-null  object 
 5   Jmlh Tahun Pendidikan  35994 non-null  int64  
 6   Status Perkawinan      35994 non-null  object 
 7   Pekerjaan              35994 non-null  object 
 8   Jenis Kelamin          35994 non-null  object 
 9   Keuntungan Kapital     35994 non-null  float64
 10  Kerugian Capital       35994 non-null  float64
 11  Jam per Minggu         35994 non-null  float64
 12  Gaji                   35994 non-null  object 
dtypes: float64(3), int64(4), object(6)
memory usage: 3.6+ MB


In [4]:
pd.options.display.max_rows = 50

In [5]:
train_df.head(50)

,id,Umur,Kelas Pekerja,Berat Akhir,Pendidikan,Jmlh Tahun Pendidikan,Status Perkawinan,Pekerjaan,Jenis Kelamin,Keuntungan Kapital,Kerugian Capital,Jam per Minggu,Gaji
0,0,21,Wiraswasta,242912,SMA,9,Belum Pernah Menikah,Servis Lainnya,Perempuan,0.0,0.0,35.0,<=7jt
1,1,49,Wiraswasta,140782,10th,6,Cerai,Ekesekutif Managerial,Perempuan,0.0,0.0,40.0,<=7jt
2,2,44,Wiraswasta,120057,D3,12,Menikah,Ekesekutif Managerial,Laki2,61404000.0,0.0,45.0,>7jt
3,3,24,Wiraswasta,194630,Sarjana,13,Belum Pernah Menikah,Spesialis,Laki2,0.0,0.0,35.0,<=7jt
4,4,33,Wiraswasta,219619,Master,14,Menikah,Spesialis,Laki2,210336000.0,0.0,40.0,>7jt
5,5,31,Wiraswasta,331065,Sarjana,13,Belum Pernah Menikah,Ekesekutif Managerial,Perempuan,0.0,19712000.0,40.0,<=7jt
6,6,32,Wiraswasta,111520,Pendidikan Tinggi,10,Belum Pernah Menikah,Servis Lainnya,Laki2,0.0,0.0,34.0,<=7jt
7,7,20,Wiraswasta,115057,SMA,9,Belum Pernah Menikah,Perbaikan Kerajinan,Laki2,0.0,0.0,40.0,<=7jt
8,8,23,?,191910,Pendidikan Tinggi,10,Belum Pernah Menikah,?,Laki2,0.0,0.0,40.0,<=7jt
9,9,43,Wiraswasta,134120,SMA,9,Cerai,Sales,Perempuan,0.0,0.0,46.0,<=7jt


In [6]:
for kolom in train_df.columns:
    print(train_df[kolom].unique())

[    0     1     2 ... 35991 35992 35993]
[21 49 44 24 33 31 32 20 23 43 28 45 52 27 26 56 59 41 51 68 64 37 67 34
 42 25 50 29 40 35 22 19 47 66 81 36 55 46 58 61 17 18 74 30 57 38 60 62
 39 70 63 90 53 48 75 71 76 82 73 72 54 65 77 84 79 88 80 69 78 87 85 83
 86]
['Wiraswasta' '?' 'Pemerintah Lokal' 'Pekerja Bebas Perusahaan'
 'Pemerintah Negara' 'Pekerja Bebas Bukan Perusahan' 'Pemerintah Provinsi'
 'Tidak Pernah Bekerja' 'Tanpa di Bayar']
[242912 140782 120057 ... 248776 148169 264722]
['SMA' '10th' 'D3' 'Sarjana' 'Master' 'Pendidikan Tinggi' '1st-4th'
 'Sekolah Professional' '7th-8th' 'Doktor' '5th-6th' '11th' '9th' 'D4'
 '12th' 'SD']
[ 9  6 12 13 14 10  2 15  4 16  3  7  5 11  8  1]
['Belum Pernah Menikah' 'Cerai' 'Menikah' 'Janda' 'Berpisah' 'Menikah LDR']
['Servis Lainnya' 'Ekesekutif Managerial' 'Spesialis'
 'Perbaikan Kerajinan' '?' 'Sales' 'Pembersih' 'Pemuka Agama' 'Petani'
 'Tech-support' 'Mesin Inspeksi' 'Supir' 'Asisten Rumah Tangga' 'Penjaga'
 'Tentara']
['Perempuan' 'L

In [7]:
for kolom in train_df.columns:
    print(kolom, train_df[train_df[kolom]=='?'][kolom].count())

id 0
Umur 0
Kelas Pekerja 2063
Berat Akhir 0
Pendidikan 0
Jmlh Tahun Pendidikan 0
Status Perkawinan 0
Pekerjaan 2069
Jenis Kelamin 0
Keuntungan Kapital 0
Kerugian Capital 0
Jam per Minggu 0
Gaji 0


C:\Users\HP\anaconda3\envs\cobaconda\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


In [8]:
train_df['Kelas Pekerja'].unique()

array(['Wiraswasta', '?', 'Pemerintah Lokal', 'Pekerja Bebas Perusahaan',
       'Pemerintah Negara', 'Pekerja Bebas Bukan Perusahan',
       'Pemerintah Provinsi', 'Tidak Pernah Bekerja', 'Tanpa di Bayar'],
      dtype=object)

In [9]:
train_df['Pekerjaan'].unique()

array(['Servis Lainnya', 'Ekesekutif Managerial', 'Spesialis',
       'Perbaikan Kerajinan', '?', 'Sales', 'Pembersih', 'Pemuka Agama',
       'Petani', 'Tech-support', 'Mesin Inspeksi', 'Supir',
       'Asisten Rumah Tangga', 'Penjaga', 'Tentara'], dtype=object)

## Ganti Nilai '?'

In [10]:
train_df['Kelas Pekerja']=train_df['Kelas Pekerja'].str.replace('?', 'Lain-lain')
train_df['Pekerjaan']=train_df['Pekerjaan'].str.replace('?', 'Lain-lain')

In [11]:
for kolom in train_df.columns:
    print(kolom, train_df[train_df[kolom]=='?'][kolom].count())

id 0
Umur 0
Kelas Pekerja 0
Berat Akhir 0
Pendidikan 0
Jmlh Tahun Pendidikan 0
Status Perkawinan 0
Pekerjaan 0
Jenis Kelamin 0
Keuntungan Kapital 0
Kerugian Capital 0
Jam per Minggu 0
Gaji 0


In [12]:
train_df['Gaji']=train_df['Gaji'].map({'<=7jt':0,'>7jt':1})
train_df.drop(['id','Berat Akhir', 'Keuntungan Kapital', 'Kerugian Capital', 'Pendidikan'], axis=1, inplace=True)
train_df.head()

,Umur,Kelas Pekerja,Jmlh Tahun Pendidikan,Status Perkawinan,Pekerjaan,Jenis Kelamin,Jam per Minggu,Gaji
0,21,Wiraswasta,9,Belum Pernah Menikah,Servis Lainnya,Perempuan,35.0,0
1,49,Wiraswasta,6,Cerai,Ekesekutif Managerial,Perempuan,40.0,0
2,44,Wiraswasta,12,Menikah,Ekesekutif Managerial,Laki2,45.0,1
3,24,Wiraswasta,13,Belum Pernah Menikah,Spesialis,Laki2,35.0,0
4,33,Wiraswasta,14,Menikah,Spesialis,Laki2,40.0,1


In [13]:
#src cat_data: http://theprofessionalspoint.blogspot.com/2019/03/how-to-separate-numeric-and-categorical.html

#Why and when shall we use One-Hot Encoding?
#One of the major problems with machine learning is that a lot of algorithms cannot work directly with categorical data. 
#Categorical data [1] are variables that can take on one of a limited number of possible values. 
#src: https://towardsdatascience.com/what-is-one-hot-encoding-and-how-to-use-pandas-get-dummies-function-922eb9bd4970

#Diselesaikan dengan One-Hot encoding tetapi performa hanya sampai 0.77 di Kaggle.
categorical_columns = train_df.select_dtypes(exclude=np.number).columns

df2 = pd.get_dummies(data=train_df, prefix=categorical_columns, drop_first=True)

In [14]:
pd.options.display.max_columns = 50
df2.head()

,Umur,Jmlh Tahun Pendidikan,Jam per Minggu,Gaji,Kelas Pekerja_Pekerja Bebas Bukan Perusahan,Kelas Pekerja_Pekerja Bebas Perusahaan,Kelas Pekerja_Pemerintah Lokal,Kelas Pekerja_Pemerintah Negara,Kelas Pekerja_Pemerintah Provinsi,Kelas Pekerja_Tanpa di Bayar,Kelas Pekerja_Tidak Pernah Bekerja,Kelas Pekerja_Wiraswasta,Status Perkawinan_Berpisah,Status Perkawinan_Cerai,Status Perkawinan_Janda,Status Perkawinan_Menikah,Status Perkawinan_Menikah LDR,Pekerjaan_Ekesekutif Managerial,Pekerjaan_Lain-lain,Pekerjaan_Mesin Inspeksi,Pekerjaan_Pembersih,Pekerjaan_Pemuka Agama,Pekerjaan_Penjaga,Pekerjaan_Perbaikan Kerajinan,Pekerjaan_Petani,Pekerjaan_Sales,Pekerjaan_Servis Lainnya,Pekerjaan_Spesialis,Pekerjaan_Supir,Pekerjaan_Tech-support,Pekerjaan_Tentara,Jenis Kelamin_Perempuan
0,21,9,35.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1
1,49,6,40.0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,44,12,45.0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,24,13,35.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,33,14,40.0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [15]:
df2.shape

(35994, 32)

In [16]:
X = df2.drop('Gaji', axis=1)
y = df2['Gaji']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)
#df2.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in df2.columns.values]

#print(X_train)

#target = df2.iloc[:, 0]
#predictors = df2.iloc[:, 1:]

#XGBModel = XGBClassifier()
dftest = pd.read_csv('test.csv')
dftest2 = pd.read_csv('test.csv')

#checking = { 'learning_rate':[0.001, 0.01, 0.1, 0.2, 0.5], 'max_depth':[1, 2, 4, 5, 7, 9, 10], 'min_child_weight':[2, 4, 5, 6, 7, 8]}
#grid = GridSearchCV(XGBModel, n_jobs=-1, param_grid=checking)
#grid.fit(X, y)

#{'learning_rate': 0.1, 'max_depth': 5, 'min_child_weight': 2}


XGBModel2=XGBClassifier(learning_rate=0.1, max_depth= 5, min_child_weight=2)
finalmodel=XGBModel2.fit(X_train, y_train)

dftest['Kelas Pekerja']=dftest['Kelas Pekerja'].str.replace('?', 'Lain-lain')
dftest['Pekerjaan']=dftest['Pekerjaan'].str.replace('?', 'Lain-lain')

dftest.drop(['id','Berat Akhir', 'Keuntungan Kapital', 'Kerugian Capital', 'Pendidikan'], axis=1, inplace=True)

df3 = pd.get_dummies(data=dftest, prefix=['Kelas Pekerja', 'Status Perkawinan', 'Pekerjaan', 'Jenis Kelamin'], drop_first=True)



#df3.head()
print(df2.shape)
print(df3.shape)


(35994, 32)
(9599, 31)


In [17]:
X_test_CSV = df3

finalmodelpred=finalmodel.predict(X_test_CSV)

df_Final = dftest2
df_Final['Gaji'] = finalmodelpred
df_Final

final_Proba = XGBModel2.predict_proba(X_test)

print(final_Proba)

final_Proba_y=final_Proba[:,1]

[[0.6863104  0.31368962]
 [0.7323704  0.26762965]
 [0.17434734 0.82565266]
 ...
 [0.9978958  0.0021042 ]
 [0.92434955 0.07565043]
 [0.8672063  0.13279371]]


In [18]:
print(rocauc(y_test, final_Proba_y))

df_Final = df_Final[['id', 'Gaji']]
df_Final

df_Final.to_csv('submission-ivanmsiegfried-9.csv', index=False)

0.8943108123672217
